In [ ]:
job_worker_name = "g_i"

In [ ]:
#@title Pip Installs
!pip uninstall -y fashion_nets
!pip install -q git+https://github.com/NiklasHoltmeyer/FashionNets.git

In [ ]:
#@title Init Kaggle
from fashionnets.train_jobs.jobs import load_job_from_notebook_name
from fashionnets.train_jobs.jobs import load_train_job
from fashionnets.train_jobs.notebook_environments import GoogleColabEnvironment

GoogleColabEnvironment().load_kaggle()

settings = load_job_from_notebook_name(job_worker_name)
import kaggle
kaggle.api.authenticate()

settings["environment"].load_dependencies(kaggle_downloader=kaggle.api.dataset_download_files,
                                          skip_preprocess=True)  # <- doesnt work from within library

In [ ]:
#@title Prepare Dataset
additional_settings = {
    "name": settings["environment"].train_job_name,
    "target_shape": settings["input_shape"],
    "format": "triplet" if settings["is_triplet"] else "quadtruplet"
}

train_job = load_train_job(**additional_settings, **settings)
train_job = {**additional_settings, **settings, **train_job}
datasets = train_job["run"]["dataset"]
train_ds, val_ds = datasets["train"], datasets["val"]

result_uploader = settings["environment"].webdav

In [ ]:
siamese_model, init_epoch, callbacks = load_siamese_model(train_job, input_shape=train_job["input_shape"],
                                                          verbose=train_job["verbose"],
                                                          result_uploader=result_uploader)

In [ ]:
siamese_model.fit(train_ds, epochs=train_job["epochs"], validation_data=val_ds, callbacks=callbacks,
                  initial_epoch=init_epoch)